In [1]:
import numpy as np
import mlx.core as mx
import mlx.core.linalg as linalg
import platform
import psutil
import time
import os

In [2]:
def get_backend(use_gpu=False):
    """
    Select computational backend based on user preference and system capabilities.
    
    When use_gpu is True, MLX will use Metal for GPU acceleration. Otherwise,
    it will use NumPy. MLX's advantage is that it can seamlessly switch between
    CPU and GPU execution while maintaining the same API.
    """
    if use_gpu:
        try:
            mx.set_default_device(mx.gpu)
            return mx, mx.linalg, "MLX with Metal GPU"
        except Exception as e:
            print(f"Warning: Could not initialize GPU: {e}")
            print("Falling back to CPU")
            mx.set_default_device(mx.cpu)
            return np, np.linalg, "NumPy"
    else:
        return np, np.linalg, "NumPy"

In [3]:
def benchmark_qr_operations(matrix_size=5000, iterations=5, use_gpu=False):
    """
    Benchmark QR factorization using either NumPy or MLX.
    
    This benchmark compares the performance of QR factorization between NumPy
    (which uses LAPACK under the hood) and MLX (which can use either CPU or
    Metal GPU acceleration). The comparison is particularly interesting because
    both implementations are highly optimized for their respective platforms.
    """
    xp, linalg, backend_name = get_backend(use_gpu)
    
    print(f"\nSystem Information:")
    print(f"OS: {platform.system()} {platform.version()}")
    print(f"CPU: {platform.processor()}")
    print(f"Memory: {psutil.virtual_memory().total / (1024**3):.1f} GB")
    print(f"Backend: {backend_name}")
    print(f"Matrix size: {matrix_size}x{matrix_size}")
    print(f"Iterations: {iterations}")
    
    times = []
    
    # Warmup iteration to ensure fair timing
    A_warmup = np.random.randn(matrix_size, matrix_size).astype(np.float32)
    if use_gpu:
        A_warmup = mx.array(A_warmup)
        Q, R = mx.linalg.qr(A_warmup)
        _ = mx.eval((Q, R))  # Force evaluation for warmup
    else:
        _ = np.linalg.qr(A_warmup)
    
    for i in range(iterations):
        # Create new matrix for each iteration to prevent caching effects
        A = np.random.randn(matrix_size, matrix_size).astype(np.float32)
        
        start_time = time.perf_counter()
        
        if use_gpu:
            # MLX implementation with GPU acceleration
            A = mx.array(A)
            Q, R = mx.linalg.qr(A)
            # Force evaluation of the computation graph
            Q, R = mx.eval((Q, R))
        else:
            # NumPy implementation using LAPACK
            Q, R = np.linalg.qr(A)
        
        end_time = time.perf_counter()
        
        operation_time = end_time - start_time
        times.append(operation_time)
        
        print(f"Iteration {i+1}/{iterations}: {operation_time:.2f} seconds")
        
        # Optional: verify factorization accuracy on first iteration
        if i == 0 and not use_gpu:
            max_error = np.max(np.abs(A - np.dot(Q, R)))
            print(f"Max error in factorization: {max_error:.2e}")
    
    return sum(times) / len(times), times

In [ ]:
def run_full_benchmark():
    """
    Run comprehensive benchmarks comparing CPU and GPU performance.
    
    This function runs the QR factorization benchmark on both CPU (using NumPy)
    and GPU (using MLX with Metal acceleration) and provides detailed statistics
    about the performance difference.
    """
    print("\nRunning CPU benchmark (NumPy)...")
    # cpu_avg, cpu_times = benchmark_qr_operations(use_gpu=False)
    cpu_avg, cpu_times = benchmark_qr_operations(matrix_size=50, iterations=1, use_gpu=False)
    
    print("\nRunning GPU benchmark (MLX)...")
    # gpu_avg, gpu_times = benchmark_qr_operations(use_gpu=True)
    gpu_avg, gpu_times = benchmark_qr_operations(matrix_size=50, iterations=1, use_gpu=True)
    
    print("\nBenchmark Results:")
    print(f"CPU Average Time: {cpu_avg:.2f} seconds")
    print(f"GPU Average Time: {gpu_avg:.2f} seconds")
    print(f"Speedup: {cpu_avg/gpu_avg:.2f}x")
    
    print("\nDetailed Statistics:")
    print("\nCPU Times:")
    print(f"Min: {min(cpu_times):.2f}s")
    print(f"Max: {max(cpu_times):.2f}s")
    print(f"Std Dev: {np.std(cpu_times):.2f}s")
    
    print("\nGPU Times:")
    print(f"Min: {min(gpu_times):.2f}s")
    print(f"Max: {max(gpu_times):.2f}s")
    print(f"Std Dev: {np.std(gpu_times):.2f}s")

if __name__ == "__main__":
    run_full_benchmark()